# Creating a dataset with a simple format 
In this tutorial, we will walk through how to generate a dataset using a simple input format that will work with some parts of the FEMR pipeline.

First, we load in the example data in a CSV file that contains all the data. In reality, this `example.csv` can be a giant file that incldues all types of data. 

In [2]:
import os
import csv
import pandas as pd
import shutil

# Import the example dataset 
example_dat = pd.read_csv("example.csv", sep=',')

# 1. Basic input schema
The input schema is a (folder of) csv file, where each csv file has <u>at minimum</u> the following columns:

`patient_id`, `start`, `code`, and `value`

 - `patient_id` is the ID for the patient who has the event. `patient_id` must be a 64 bit unsigned integer

 - `start` is the start timestamp for an event, ideally when the event is initially recorded in the database. `start` must be an ISO 8601 timestamp string

 - `code` is a string that identifies what type of event occurred. It must consist of two parts, a vocabulary signfier and the code itself, seperated by a "/" character. For example ICD10CM/E11.4 would indicate an E11.4 ICD10 code.

 - `value` is a value associated with the event. It can either be a numeric value, an arbitrary string, or nothing.


You may also add arbitrary columns for any csv file. Those will be added to each event. The columns can vary between csv files.
We recommend adding columns to note dosage, visit_ids, and lab units, source Clarity tables, etc, etc.

The first row (in time) for each patient is considered their birth event.

Ordering of rows for each patient, or patient rows being split across files doesn't matter.
Everything will be resorted and joined as part of the ETL process (i.e. creating a `PatientDatabase` involves sorting by patient and events).

All different types of EMR data can be mapped to those four core columns. Here are the common tips for different types of fields:

Demographics should generally be mapped as as demographics codes assigned to the birth date of the patient (with no value assigned).

Labs should be assigned to when the lab result is available, with the value numeric is possible, but text valued otherwise.

Procedures and diagnosis codes should generally be mapped to when the event happened, with no value attached.

Other strange datatypes, such as flowsheets can be added as needed, with either string or numeric values as whatever is more natural.

## Exercise 1: Add demographic information
Now, we display `example_dat` that contains a single patient with three rows. 

We use patients' date of birth as the very first visit time in our data format, so the first event is always `Birth`

Rows 2 and 3 include the demographic information, `Gender` and `Race`, respectively, of this patient with corresponding values. Demographics should generally be mapped as demographics codes assigned to the birth date of the patient (with no value assigned). For demographic rows, the vocabulary signifier and the code itself are the same.

In [3]:
example_dat.loc[0:2]

,patient_id,start,code,value,units,dosage
0,3,1970-01-07,Birth/Birth,NaN,NaN,NaN
1,3,1990-01-07,Gender/Gender,Female,NaN,NaN
2,3,1990-01-07,Race/Race,White,NaN,NaN


# 2. Expanding your dataset
You now show additional rows for other events present in your EHR records.

## Exercise 2: Add diagnosis information
We now add more events/rows that capture patients' diagnosis information, e.g., ICD 9/10 codes

For diagnosis, the `code` column has two parts, the vocabulary signifier (e.g., ICD10CM) and the code itself (e.g., E11.4, E10.1, etc.) The `value` column should be left as empty. Procedures and diagnosis codes should generally be mapped to when the event happened.

Note that different diagnoses may be given at different visits, so the corresponding `start` timestamps may be different.

In [4]:
example_dat.loc[3:4]

,patient_id,start,code,value,units,dosage
3,3,2022-05-03,ICD10CM/E11.4,NaN,NaN,NaN
4,3,2022-06-05,ICD10CM/E10.1,NaN,NaN,NaN


## Exercise 3: Add lab test information
We now add more events/rows that capture patients' lab values, e.g., Vitals/Blood Pressure

For vitals, the `code` column has two parts, the vocabulary signifier (e.g., Vitals) and the code itself (e.g., Blood Pressure, HbA1c, etc.) The `value` column should contain corresponding numeric values when possible, but text valued otherwise.

We also recommend adding another column, `units`, to record the units of each test result.

Note that different lab tests may be given at different visits, so the corresponding `start` timestamps may differ.

In [5]:
example_dat.loc[5:6]

,patient_id,start,code,value,units,dosage
5,3,2020-07-09,Vitals/Blood Pressure,160,mmHg,NaN
6,3,2020-08-09,Vitals/HbA1c,7,NaN,%


## Exercise 4: Add medication information
We now add more events/rows that capture patients' medication intake info., e.g., Drug/Atorvastatin

For medications, the `code` column has two parts, the vocabulary signifier (e.g., Drug) and the code itself (e.g., Atorvastatin, Heparin Lock Flush, Multivitamins, etc.) The `value` column should be empty.

We add another column, `dosage`, to record the dose of the prescribed medication. For medications, the `units` column indicates the unit of the medication dose.

Note that different lab tests may be given at different visits, so the corresponding `start` timestamps may differ.

In [6]:
example_dat.loc[7:8]

,patient_id,start,code,value,units,dosage
7,3,2022-06-05,Drug/Atorvastatin,NaN,mg,50
8,3,2022-07-06,Drug/Multivitamins,NaN,ml,5


## Exercise 5: Add note information
We now add more events/rows that capture notes written about patients.

For notes, the main consideration is that you often need quoting and escaping in order to process notes with quote characters, commas and newlines. 

We follow the [RFC 4180 spec](https://www.loc.gov/preservation/digital/formats/fdd/fdd000323.shtml#:~:text=RFC%204180%20stipulates%20the%20use,double%20quotes%20(Hex%2022).) for escaping, which is the default format for the Python csv library.

In [7]:
example_dat.loc[9:10]

,patient_id,start,code,value,units,dosage
9,3,2022-06-05,Note/ProgressNote,Patient Bob came to the clinic today,NaN,NaN
10,3,2022-06-06,Note/ProgressNote,"Complicated notes generally need escaping , "" ...",NaN,NaN


# 3. Additional considerations
When adding columns, the goal is to add as many columns as needed to maximum the clarity and completeness of the raw/original data. 

For simplicity, we only included one patient in the above datasets, but an arbitrary number of patients can be captured in the same file by adding more rows with different `patient_id`.


## Exercise 6: Generate synthetic examples
We now create a dataset with 100 synthetic patients' data

In [7]:
import random
import datetime
import string

def add_random_patient(patient_id, f):
    epoch = datetime.date(1990, 1, 1)
    birth = epoch + datetime.timedelta(days=random.randint(100, 1000))
    current_date = birth
    
    code_cat = ["Birth","Gender", "Race","ICD10CM","CPT", "Drug", "Vitals"]
    gender_values = ["F","M"]
    race_values = ["White","Non-White"]
    index = random.randint(0,1)
    for code_type in code_cat:
        if code_type == "Birth":
            clarity = "PATIENT"
            visit_id = 1
            value = ''
            dosage = ''
            unit = ''
            f.write(f"{patient_id},{birth.isoformat()},{code_type}/{code_type},{value},{dosage},{visit_id},{unit},{clarity}\n")
        elif code_type == "Gender":
            clarity = "PATIENT"
            visit_id = 1
            value = gender_values[index]
            dosage = ''
            unit = ''
            f.write(f"{patient_id},{birth.isoformat()},{code_type}/{code_type},{value},{dosage},{visit_id},{unit},{clarity}\n")
        elif code_type == "Race":
            clarity = "PATIENT"
            visit_id = 1
            value = race_values[index]
            dosage = ''
            unit = ''
            f.write(f"{patient_id},{birth.isoformat()},{code_type}/{code_type},{value},{dosage},{visit_id},{unit},{clarity}\n") 
        else:
            for code in range(random.randint(1, 10)):
                code = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
                current_date = current_date +  datetime.timedelta(days=random.randint(0, 100))
                visit_id = random.randint(0,20)
                if code_type == "ICD10CM":
                    clarity = "DIAGNOSIS"
                    value = ''
                    dosage = ''
                    unit = ''
                elif code_type == "CPT":
                    clarity = "PROCEDURES"
                    value = ''
                    dosage = ''
                    unit = ''
                elif code_type == "Drug":
                    clarity = "MED_ORDER"
                    value = ''
                    dosage = random.randint(10,50)
                    unit = "mg"
                elif code_type == "Vitals":
                    clarity = "LAB_RESULT"
                    value = random.randint(80,200)
                    dosage = ''
                    unit = "mmHg"
                f.write(f"{patient_id},{current_date.isoformat()},{code_type}/{code},{value},{dosage},{visit_id},{unit},{clarity}\n") 


In [8]:
for file_no in range(1, 3):
    with open("many_examples.csv", "w") as f:    
        # Use the simpler 4 column minimum format, with no optional columns
        f.write("patient_id,start,code,value,dosage,visit_ids,lab_units,clarity_source\n")
        for patient_id in range(file_no * 100, (file_no + 1) * 100):
            add_random_patient(patient_id, f)
        
        if file_no == 1:
            with open(os.path.join(INPUT_DIR, f"{file_no}.csv"), newline='') as csvfile:
                reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
                for i, row in enumerate(reader):
                    print(', '.join(row))
                    if(i >= 9):
                        break

patient_id,start,code,value,dosage,visit_ids,lab_units,clarity_source
100,1990-08-09,Birth/Birth,,,1,,PATIENT
100,1990-08-09,Gender/Gender,M,,1,,PATIENT
100,1990-08-09,Race/Race,Non-White,,1,,PATIENT
100,1990-10-28,ICD10CM/DNFJ5IDL,,,4,,DIAGNOSIS
100,1990-11-19,ICD10CM/7D02JHHM,,,17,,DIAGNOSIS
100,1990-11-19,ICD10CM/SHJQD6GA,,,14,,DIAGNOSIS
100,1991-02-04,ICD10CM/0ZT3UU66,,,18,,DIAGNOSIS
100,1991-03-28,ICD10CM/TC7ZWA0S,,,5,,DIAGNOSIS
100,1991-06-16,ICD10CM/27NHK4XA,,,13,,DIAGNOSIS


# Additional notes
Ordering of rows for each patient, or patient rows being split across files doesn't matter. Everything will be resorted and joined as part of the ETL process.

Other atypical datatypes, such as flowsheets can be added as needed, with either string or numeric values as whatever is more natural.

## 4. Convert the directory to an extract
We now convert the dataset we created above to an extract using the function [etl_simple_femr](https://github.com/som-shahlab/femr/blob/main/src/femr/etl_pipelines/simple.py#L66) from the femr repo

We need to first create folders to save the dataset and associated files 

In [21]:
TARGET_DIR = "trash/simple_femr"

if os.path.exists(TARGET_DIR):
    shutil.rmtree(TARGET_DIR)

os.makedirs(TARGET_DIR)

INPUT_DIR = os.path.join(TARGET_DIR, "simple_input")
os.makedirs(INPUT_DIR)

We now move/copy the `example.csv` into the `INPUT_DIR` folder

The output extract is a femr [PatientDatabase](https://github.com/som-shahlab/femr/blob/Miking98-patch-1/tutorials/0_How%20FEMR%20Works%20%2B%20Toy%20Example.ipynb) that can be directly used by the femr pipeline

In [22]:
# Create directories for storing the extract and extract log
LOG_DIR = os.path.join(TARGET_DIR, "logs")
EXTRACT_DIR = os.path.join(TARGET_DIR, "extract")

import femr
import femr.etl_pipelines.simple
os.system(f"etl_simple_femr {INPUT_DIR} {EXTRACT_DIR} {LOG_DIR} --num_threads 2")

Done with main 2023-04-20T17:09:28.48974651+00:00
Done with meta 2023-04-20T17:09:28.489844269+00:00
Converting to extract 2023-04-20 17:09:28.475023


2023-04-20 17:09:28,436 [MainThread  ] [INFO ]  Extracting from OMOP with arguments Namespace(simple_source='trash/simple_femr/simple_input', target_location='/local-scratch/nigam/projects/yizhex/femr/tutorials/trash/simple_femr/extract', temp_location='/local-scratch/nigam/projects/yizhex/femr/tutorials/trash/simple_femr/logs', num_threads=2)
2023-04-20 17:09:28,436 [MainThread  ] [INFO ]  Converting to events
2023-04-20 17:09:28,460 [MainThread  ] [INFO ]  Converting to patients
2023-04-20 17:09:28,474 [MainThread  ] [INFO ]  Converting to extract


0

# 5. Open and view the data
We now open and take a look at the femr extract we generated in the last step using the function [PatientDatabase](https://github.com/som-shahlab/femr/blob/main/src/femr/extension/datasets.pyi#L24) from the femr repo

In [24]:
import femr.datasets

database = femr.datasets.PatientDatabase(EXTRACT_DIR)

# Number of patients
print("Num patients", len(database))

# Print out patient_id 0 (the first example patient we created)
patient = database[0]
print(patient)

# Also note that concepts have been mapped to integers
print("What code 3 means", database.get_code_dictionary()[3])  # Returns what code=3 means in the database

# You can pull things like dosage by looking at the event
for event in patient.events:
    print(event, event.dosage)

Num patients 1
Patient(patient_id=0, events=(Event(start=1970-01-07 00:00:00, code=6, units=, dosage=), Event(start=1990-01-07 00:00:00, code=0, value=Female, units=, dosage=), Event(start=1990-01-07 00:00:00, code=8, value=White, units=, dosage=), Event(start=2020-07-09 00:00:00, code=9, value=160.0, units=mmHg, dosage=), Event(start=2020-08-09 00:00:00, code=7, value=7.0, units=, dosage=%), Event(start=2022-05-03 00:00:00, code=5, units=, dosage=), Event(start=2022-06-05 00:00:00, code=3, value=Patient Bob came to the clinic today, units=, dosage=), Event(start=2022-06-05 00:00:00, code=4, units=, dosage=), Event(start=2022-06-05 00:00:00, code=2, units=mg, dosage=50), Event(start=2022-07-06 00:00:00, code=1, units=ml, dosage=5)))
What code 3 means Note/ProgressNote
Event(start=1970-01-07 00:00:00, code=6, units=, dosage=) 
Event(start=1990-01-07 00:00:00, code=0, value=Female, units=, dosage=) 
Event(start=1990-01-07 00:00:00, code=8, value=White, units=, dosage=) 
Event(start=2020-